In [1]:
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from torch import Tensor

from src.common.paths import PlotPaths
from src.inverse_problems.footprints.footprint_loader import load_gaussian_plume_footprint
from src.inverse_problems.footprints.gaussian_plume_model import GaussianPlumeModel

In [2]:
animation_paths = PlotPaths.ANIMATIONS / "footprints"
animation_paths.mkdir(exist_ok=True)

In [3]:
def animate(sensing_matrix_: Tensor, path: Path) -> None:    
    fig, ax = plt.subplots()

    image = ax.imshow(sensing_matrix_[0].reshape(32, 32), vmin=0, vmax=vmax)
    
    plt.colorbar(image, ax=ax, label=r"$\text{ppm} \cdot \text{m}^2 \cdot \text{s} \cdot \mu\text{mol}^{-1}$")
    plt.tight_layout()
    
    def update(frame):
        image.set_array(sensing_matrix_[frame].reshape(32, 32))
        return [image]
    
    ani = FuncAnimation(fig, update, frames=len(sensing_matrix_), blit=True, interval=100)
    
    plt.close()
    
    ani.save(path, writer="pillow", fps=30)

# My Implementation

In [4]:
sensor_locations = [
    (4, 5),
    (25, 6),
    (5, 26),
    (26, 25),
    (14, 15),
    (6, 15),
    (14, 5),
    (24, 15),
    (15, 27),
    (10, 9),
    (11, 20),
    (21, 10),
    (19, 21),
]

measurements_per_sensor = 50

In [5]:
model = GaussianPlumeModel()

footprints = []

for sensor_x, sensor_y in sensor_locations:
    footprints += model.get_sensitivities_for_sensor(sensor_x, sensor_y, num_measurements=measurements_per_sensor)

In [6]:
sensing_matrix = np.zeros((len(footprints), 32 * 32))
for i, footprint in enumerate(footprints):
    sensing_matrix[i, :] = footprint.reshape(32 * 32)

vmax = sensing_matrix.max()

In [7]:
animate(sensing_matrix_=sensing_matrix, path=animation_paths / "my_footprints.gif")

# Footprints by Zanger et al.

In [8]:
num_measurement_stations = 50

sensing_matrix_zanger = Tensor(load_gaussian_plume_footprint(num_measurement_stations))

vmax = sensing_matrix_zanger.max()

In [9]:
animate(sensing_matrix_=sensing_matrix_zanger, path=animation_paths / "zanger_footprints.gif")